In [2]:
#Path to your relaxed pdb
pdb = '/content/RELAXED_GPCR-PEPTIDE_PREDICTED_STRUCTURE.pdb'

#RUN

In [3]:
pip install pdbe-arpeggio # Successfully installed pdbe-arpeggio-1.4.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.3 MB/s eta 0:00:00


In [4]:
pip install atomium #  atomium-1.0.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 35.7 MB/s eta 0:00:00


In [5]:
import sys, os
from sys import version_info


In [6]:
PYTHON_VERSION = f"{version_info.major}.{version_info.minor}"

print("installing conda...")
os.system("wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh")
os.system("bash Miniforge3-Linux-x86_64.sh -bfp /usr/local")
os.system("mamba config --set auto_update_conda false")
os.system("touch CONDA_READY")

#  os.system(f"mamba install -y -q -c conda-forge openmm=7.7.0 python='{PYTHON_VERSION}' pdbfixer 2>&1 1>/dev/null")

PATH = f"/usr/local/lib/python{PYTHON_VERSION}/site-packages/"
if PATH not in sys.path:
  sys.path.insert(0, PATH)

installing conda...


In [7]:
os.system(f"mamba install -y -q -c conda-forge python='{PYTHON_VERSION}' 2>&1 1>/dev/null")

0

In [8]:
os.system(f"mamba install -y -q -c conda-forge gemmi openbabel biopython 2>&1 1>/dev/null")

0

In [9]:
from arpeggio.core import InteractionComplex
from arpeggio.core.utils import max_mem_usage
import atomium
import glob
import pandas as pd
import time


In [10]:
section_to_add = '''loop_
_chem_comp.id
_chem_comp.type
_chem_comp.mon_nstd_flag
_chem_comp.name
_chem_comp.pdbx_synonyms
_chem_comp.formula
_chem_comp.formula_weight
ALA "L-peptide linking" y ALANINE         ? "C3 H7 N O2"     89.093
ARG "L-peptide linking" y ARGININE        ? "C6 H15 N4 O2 1" 175.209
ASN "L-peptide linking" y ASPARAGINE      ? "C4 H8 N2 O3"    132.118
ASP "L-peptide linking" y "ASPARTIC ACID" ? "C4 H7 N O4"     133.103
CYS "L-peptide linking" y CYSTEINE        ? "C3 H7 N O2 S"   121.158
GLN "L-peptide linking" y GLUTAMINE       ? "C5 H10 N2 O3"   146.144
GLU "L-peptide linking" y "GLUTAMIC ACID" ? "C5 H9 N O4"     147.129
GLY "peptide linking"   y GLYCINE         ? "C2 H5 N O2"     75.067
HIS "L-peptide linking" y HISTIDINE       ? "C6 H10 N3 O2 1" 156.162
ILE "L-peptide linking" y ISOLEUCINE      ? "C6 H13 N O2"    131.173
LEU "L-peptide linking" y LEUCINE         ? "C6 H13 N O2"    131.173
LYS "L-peptide linking" y LYSINE          ? "C6 H15 N2 O2 1" 147.195
MET "L-peptide linking" y METHIONINE      ? "C5 H11 N O2 S"  149.211
PHE "L-peptide linking" y PHENYLALANINE   ? "C9 H11 N O2"    165.189
PRO "L-peptide linking" y PROLINE         ? "C5 H9 N O2"     115.130
SER "L-peptide linking" y SERINE          ? "C3 H7 N O3"     105.093
THR "L-peptide linking" y THREONINE       ? "C4 H9 N O3"     119.119
TRP "L-peptide linking" y TRYPTOPHAN      ? "C11 H12 N2 O2"  204.225
TYR "L-peptide linking" y TYROSINE        ? "C9 H11 N O3"    181.189
VAL "L-peptide linking" y VALINE          ? "C5 H11 N O2"    117.146
#
loop_
_atom_site.group_PDB
_atom_site.id'''

to_replace = '''loop_
_atom_site.group_PDB
_atom_site.id'''

In [11]:
selections = ["/B//"]  # /<chain_id>/<res_num>[<ins_code>]/<atom_name>
interacting = 5.0
ph = 7.4
vdw_comp = 0.1
include_sequence_adjacent = False

try:
    pdb1 = atomium.open(pdb)
    pdb1.model.save('current.cif')

    with open('current.cif', 'r') as opened:
        ok = opened.read()
        ok = ok.replace(to_replace, section_to_add)
    with open('currentsurgery.cif', 'w') as opened:
        opened.write(ok)

    i_complex = InteractionComplex(
        'currentsurgery.cif', vdw_comp, interacting, ph
    )
    i_complex.structure_checks()
    i_complex.initialize()
    i_complex.run_arpeggio(
        selections, interacting, vdw_comp, include_sequence_adjacent
    )

    contacts = i_complex.get_contacts()
    contacts = pd.DataFrame(contacts)
    interonly = contacts[contacts['interacting_entities'] == 'INTER']
    interonly['contact'] = interonly['contact'].apply(lambda x: tuple(x))
    noproximal = interonly[interonly['contact'] != ('proximal',)].reset_index(drop=True)
    noproximal['path'] = pdb
    noproximal.to_parquet('/content/single_contact.parquet')

except Exception as e:
    print(f"Failed to process: {e}")


<ipython-input-11-597a2e4c76dc>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interonly['contact'] = interonly['contact'].apply(lambda x: tuple(x))
